In [24]:
import pandas as pd, json, random

SRC = "../online_shopping_10_cats.csv"
OUT = "sft_100.jsonl"     # 迷你训练集

label_map = {0:"负面", 1:"正面", 2:"中性"}

sys_prompt = "你是电商评论情感分析助手，必须输出：情感（正面/负面/中性）+ 原因解释"

df = pd.read_csv(SRC)

# 随机抽 100 条（如果总数据不足 100，会自动取全部）
random.seed(42)
df = df.sample(n=min(100, len(df)))

with open(OUT, "w", encoding="utf-8") as f:
    for i, r in df.iterrows():
        emo  = label_map[int(r["label"])]
        text = str(r["review"]).strip()
        ans  = f"情感：{emo}\n原因:该评论表达{emo}态度"

        obj = {
            "messages":[
                {"role": "system",    "content": sys_prompt},
                {"role": "user",      "content": f"评论：{text}"},
                {"role": "assistant", "content": ans}
            ]
        }

        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print(f"生成 100 条 SFT 训练样本 → {OUT}")


生成 100 条 SFT 训练样本 → sft_100.jsonl


In [ ]:
import json
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model



# 1. 基本环境配置
MODEL = "Qwen/Qwen2.5-1.5B"     # 基座模型
DATA = "sft_100.jsonl"          # 你的 SFT 数据
OUT = r"C:\Users\jh\Desktop\LLM\sft_out"   # LoRA 输出目录



# 2. SFT 数据集加载器
class SFTDataset(Dataset):
    def __init__(self, path):
        self.items = []
        for l in open(path, "r", encoding="utf-8"):
            o = json.loads(l)
            text = ""
            for m in o["messages"]:
                role = {
                    "system": "系统",
                    "user": "用户",
                    "assistant": "助手"
                }[m["role"]]
                text += f"{role}：{m['content']}\n"
            self.items.append(text)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, i):
        return self.items[i]



# 3. 定义 collate 函数
def collate(batch):
    enc = tok(batch, return_tensors="pt", padding=True, truncation=True)
    enc["labels"] = enc["input_ids"].clone()
    enc["labels"][enc["labels"] == tok.pad_token_id] = -100   # 忽略 pad
    return enc



# 4. 加载 tokenizer
tok = AutoTokenizer.from_pretrained(MODEL)
tok.pad_token = tok.eos_token



# 5. 加载模型（CPU）
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float32,
    device_map={"": "cpu"}    # 强制使用 CPU
)



# 6. 注入 LoRA 结构
model = get_peft_model(
    model,
    LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        task_type="CAUSAL_LM"
    )
)
model.print_trainable_parameters()



# 7. Trainer 配置
trainer = Trainer(
    model=model,
    train_dataset=SFTDataset(DATA),
    data_collator=collate,
    args=TrainingArguments(
        output_dir=OUT,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        num_train_epochs=1,
        logging_steps=20,
        no_cuda=True   # 禁用 GPU
    )
)


# 8. 训练
trainer.train()

model.save_pretrained(OUT)
tok.save_pretrained(OUT)

print("\n🎯 CPU LoRA SFT 训练完成 →", OUT)


`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


c:\Users\jh\anaconda3\Lib\site-packages\transformers\training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss



🎯 CPU LoRA SFT 训练完成 → C:\Users\jh\Desktop\LLM\sft_out


In [ ]:
import faiss, pandas as pd, numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

ENC = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
SRC = "../online_shopping_10_cats.csv"       
OUT_IDX = "reviews.index"
OUT_TXT = "reviews.parquet"

df = pd.read_csv(SRC)
texts = df["review"].astype(str).tolist()

model = SentenceTransformer(ENC)

vecs = []
for i in tqdm(range(0, len(texts), 256)):
    emb = model.encode(texts[i:i+256], normalize_embeddings=True)
    vecs.append(emb)

vecs = np.vstack(vecs).astype("float32")

index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)

faiss.write_index(index, OUT_IDX)
df.to_parquet(OUT_TXT)

print("⚡ FAISS 检索库已构建！")


100%|██████████| 246/246 [10:16<00:00,  2.51s/it]


⚡ FAISS 检索库已构建！


In [ ]:
import faiss, torch, pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE = "Qwen/Qwen2.5-1.5B"
LORA = r"C:\Users\jh\Desktop\LLM\sft_out"
IDX  = "reviews.index"
TXT  = "reviews.parquet"


#  Load tokenizer & model
tok = AutoTokenizer.from_pretrained(LORA)
base = AutoModelForCausalLM.from_pretrained(BASE, torch_dtype=torch.float32)
model = PeftModel.from_pretrained(base, LORA).eval()

#  Load retriever
encoder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
index = faiss.read_index(IDX)
df = pd.read_parquet(TXT)


def ask(q):
    # 1️ 检索相似评论
    emb = encoder.encode([q], normalize_embeddings=True).astype("float32")
    _, I = index.search(emb, 5)
    evid = [df.iloc[i]["review"] for i in I[0]]

    # 2️ 构建 prompt
    prompt = f"""
系统：你是情感分析助手，需要结合证据判断情绪。
用户评论：{q}

相似评论证据：
{chr(10).join("- "+e for e in evid)}

输出格式必须为：
情感：<正面/负面/中性>
原因：<一句话解释>
助手：
"""

    out = model.generate(
        **tok(prompt, return_tensors="pt"),
        max_new_tokens=80,
        do_sample=False
    )

    print("\n==== RESULT ====")
    print(tok.decode(out[0], skip_special_tokens=True))


ask("这个手机真的太卡了，用起来很失望！")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\jh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jh\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



==== RESULT ====

系统：你是情感分析助手，需要结合证据判断情绪。
用户评论：这个手机真的太卡了，用起来很失望！

相似评论证据：
- 好差劲的手机
- 手机真是烂，卡的一匹
- 好废的手机，不会再买了
- 很遗憾包装很不好，感觉有人拆开过！希望手机是正品的。
- 好好玩手机，现在真不行了，也就那样，用着，不是很好。

输出格式必须为：
情感：<正面/负面/中性>
原因：<一句话解释>
助手：
情感：负面
原因：手机真的太卡了，用起来很失望！
助手：


In [ ]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


# 1️ 准备测试集
df = pd.read_csv("../online_shopping_10_cats.csv")

random.seed(42)
test_df = df.sample(n=200)

test_texts = test_df["review"].astype(str).tolist()
test_labels = test_df["label"].tolist()



# 2️ 原始模型预测（BASE）
BASE = "Qwen/Qwen2.5-1.5B"
tok_base = AutoTokenizer.from_pretrained(BASE)
model_base = AutoModelForCausalLM.from_pretrained(BASE)

def predict_base(text):
    prompt = f"评论：{text}\n请判断情感是正面、负面还是中性。只输出情感类别。"
    out = model_base.generate(
        **tok_base(prompt, return_tensors="pt"),
        max_new_tokens=20,
        do_sample=False
    )
    ans = tok_base.decode(out[0], skip_special_tokens=True)
    if "正面" in ans:
        return 1
    if "负面" in ans:
        return 0
    return 2



# 3️ 微调后模型预测（LoRA SFT）
LORA = r"C:\Users\jh\Desktop\LLM\sft_out"

tok_ft = AutoTokenizer.from_pretrained(LORA)
base_ft = AutoModelForCausalLM.from_pretrained(BASE)
model_ft = PeftModel.from_pretrained(base_ft, LORA).eval()

def predict_ft(text):
    prompt = f"系统：你是情感分析助手。\n用户评论：{text}\n请输出情感。"
    out = model_ft.generate(
        **tok_ft(prompt, return_tensors="pt"),
        max_new_tokens=20,
        do_sample=False
    )
    ans = tok_ft.decode(out[0], skip_special_tokens=True)
    if "正面" in ans:
        return 1
    if "负面" in ans:
        return 0
    return 2



# 4️ accuracy 对比
base_preds = []
ft_preds = []

for text in tqdm(test_texts):
    base_preds.append(predict_base(text))
    ft_preds.append(predict_ft(text))

base_acc = (np.array(base_preds) == np.array(test_labels)).mean()
ft_acc = (np.array(ft_preds) == np.array(test_labels)).mean()

print("原始模型 accuracy =", base_acc)
print("微调后模型 accuracy =", ft_acc)
print("提升 =", ft_acc - base_acc)


  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
  0%|          | 1/200 [00:21<1:12:42, 21.92s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
  1%|          | 2/200 [00:48<1:21:44, 24.77s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
  2%|▏         | 3/200 [00:54<53:10, 16.20s/it]  Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
  2%|▏         | 4/200 [01:01<40:59, 12.55s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
  2%|▎         | 5/200 [01:13<40:15, 

原始模型 accuracy = 0.5
微调后模型 accuracy = 0.6
提升 = 0.09999999999999998
